In [1]:
import torch
import torch.nn as nn

In [ ]:
# word form -> (morphology, orthography, phonology and multi word expressions)
#       properties of the word and characters themselvies. ie. silent letters, sounds like, pre/suffix

# PhonologicalEncoder() To-do: phonemes, (phonology)

# OrthographicEncoder() To-do: graphemes, (orthography)

# MorphologicalEncoder() To-do: morphemes, (morphology)

# LexicalEncoder() To-do: lemmas, (lexicon)

# SemanticEncoder() done: synonyms, hyponyms/hypernyms,(homographs)
        # To-do:  meronyms/holonyms, polysemy (maybe?)

# SyntacticEncoder() To-do: word order, word class, (morphology)

# PragmaticEncoder() To-do: implicature, presupposition, (conversational implicature)

# DiscourseEncoder() To-do: coherence, cohesion, (anaphora)

# WorldEncoder() To-do: encyclopedic, (associations)

# Combinatorial() To-do: combinations



In [ ]:
prompt = """Solve today’s NYT Connections game. Here are the instructions for how to play this game:
Find groups of four items that share something in common.
Category Examples:
FISH: Bass, Flounder, Salmon, Trout
FIRE ___: Ant, Drill, Island, Opal
Categories will always be more specific than
‘5-LETTER-WORDS’, ‘NAMES’, or ‘VERBS.’
Example 1:
Words: [‘DART’, ‘HEM’, ‘PLEAT’, ‘SEAM’,
‘CAN’, ‘CURE’, ‘DRY’, ‘FREEZE’, ‘BITE’,
‘EDGE’, ‘PUNCH’, ‘SPICE’, ‘CONDO’, ‘HAW’,
‘HERO’, ‘LOO’]
Groupings:
1. Things to sew: [‘DART’, ‘HEM’, ‘PLEAT’,
‘SEAM’]
2. Ways to preserve food: [‘CAN’, ‘CURE’,
‘DRY’, ‘FREEZE’]
3. Sharp quality: [‘BITE’, ‘EDGE’, ‘PUNCH’,
‘SPICE’]
4. Birds minus last letter: [‘CONDO’, ‘HAW’,
‘HERO’, ‘LOO’]
Example 2:
Words: [1COLLECTIVE’, ‘COMMON’, ‘JOINT’,
‘MUTUAL’, ‘CLEAR’, ‘DRAIN’, ‘EMPTY’,
‘FLUSH’, ‘CIGARETTE’, ‘PENCIL’, ‘TICKET’,
‘TOE’, ‘AMERICAN’, ‘FEVER’, ‘LUCID’,
‘PIPE’]
Groupings:
1. Shared: [‘COLLECTIVE’, ‘COMMON’,
‘JOINT’, ‘MUTUAL’]
2. Rid of contents: [‘CLEAR’, ‘DRAIN’,
‘EMPTY’, ‘FLUSH’]
3. Associated with “stub”: [‘CIGARETTE’,
‘PENCIL’, ‘TICKET’, ‘TOE’]
4. __ Dream: [‘AMERICAN’, ‘FEVER’, ‘LU-
CID’, ‘PIPE’])
Example 3:
Words: [‘HANGAR’, ‘RUNWAY’, ‘TARMAC’,
‘TERMINAL’, ‘ACTION’, ‘CLAIM’, ‘COM-
PLAINT’, ‘LAWSUIT’, ‘BEANBAG’, ‘CLUB’,
‘RING’, ‘TORCH’, ‘FOXGLOVE’, ‘GUMSHOE’,
‘TURNCOAT’, ‘WINDSOCK’]
Groupings:
1. Parts of an airport: [‘HANGAR’, ‘RUNWAY’,
‘TARMAC’, ‘TERMINAL’]
2. Legal terms: [‘ACTION’, ‘CLAIM’, ‘COM-
PLAINT’, ‘LAWSUIT’]
3. Things a juggler juggles: [‘BEANBAG’,
‘CLUB’, ‘RING’, ‘TORCH’]
4. Words ending in clothing: [‘FOXGLOVE’,
‘GUMSHOE’, ‘TURNCOAT’, ‘WIND-
SOCK’]
Categories share commonalities:
• There are 4 categories of 4 words each
• Every word will be in only 1 category
• One word will never be in two categories
• As the category number increases, the connec-
tions between the words and their category
become more obscure. Category 1 is the most
easy and intuitive and Category 4 is the hard-
est
• There may be a red herrings (words that seems
to belong together but actually are in separate
categories)
• Category 4 often contains compound words
with a common prefix or suffix word
• A few other common categories include word
and letter patterns, pop culture clues (such as
music and movie titles) and fill-in-the-blank
phrases
You will be given a new example (Example 4) with
today’s list of words. First explain your reason
for each category and then give your final answer
following the structure below (Replace Category 1,
2, 3, 4 with their names instead)
Groupings:
Category1: [word1, word2, word3, word4]
Category2: [word5, word6, word7, word8]
Category3: [word9, word10, word11, word12]
Category4: [word13, word14, word15, word16]
Remember that the same word cannot be re-
peated across multiple categories, and you need
to output 4 categories with 4 distinct words each.
Also do not make up words not in the list. This is
the most important rule. Please obey
Example 4:
Words : [InsertGame]
Grouping"""

In [ ]:
# example 
# Words: [‘DART’, ‘HEM’, ‘PLEAT’, ‘SEAM’, ‘CAN’, ‘CURE’, ‘DRY’, ‘FREEZE’, ‘BITE’, ‘EDGE’, ‘PUNCH’, ‘SPICE’, ‘CONDO’, ‘HAW’, ‘HERO’, ‘LOO’]
# Groupings:
# 1. Things to sew: [‘DART’, ‘HEM’, ‘PLEAT’, ‘SEAM’]
# 2. Ways to preserve food: [‘CAN’, ‘CURE’, ‘DRY’, ‘FREEZE’]
# 3. Sharp quality: [‘BITE’, ‘EDGE’, ‘PUNCH’, ‘SPICE’]
# 4. Birds minus last letter: [‘CONDO’, ‘HAW’, ‘HERO’, ‘LOO’]


In [35]:
import nltk
from nltk.corpus import wordnet as wn
import json
from itertools import combinations

nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /home/auschra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
with open('datasets/history.json' , 'r') as f:
    history = json.load(f)

min_similarity = 0.7


for entry in history[:1]:
    top_pairs = {}
    groups = entry['answers']       # entry is a single game
    words = [word for group in groups for word in group['members']]    # get all words 
    words = [word.lower() for word in words]
    print(words)

    for i, word in enumerate(words):        # iterate over all words
        syns1 = wn.synsets(word)            # get all synsets of the word
        #print(f"\nWord '{word}' has {len(syns1)} synsets")

                
        for j, other_word in enumerate(words):   # iterate over all words again
                    if i == j:
                        continue  # dont compare with same word

                    syns2 = wn.synsets(other_word)
                    highest_cossim = 0

                    # sim between synsets of word and other_word
                    for syn1 in syns1:
                        for syn2 in syns2:
                            cossim = syn1.wup_similarity(syn2)
                            if cossim is not None and cossim > highest_cossim:
                                highest_cossim = cossim

                        # Store pairs with high similarity
                        if highest_cossim >= min_similarity:
                            top_pairs[(word, other_word)] = highest_cossim
                            #print(f"Pair ({word}, {other_word}) has similarity {highest_cossim}")


    # find groups of 4 words with high similarity
    high_similarity_words = list(set([w for pair in top_pairs.keys() for w in pair]))

    # check all combinations of 4 words
    for word_group in combinations(high_similarity_words, 4):
        pairs = list(combinations(word_group, 2))
        if all(pair in top_pairs for pair in pairs):
            print("\nFound set of 4 highly similar words:", word_group)
            break
    else:
        print("\nNo set of 4 words found with high similarity across all pairs.")

In [41]:
class Semantics():
    def __init__(self):
        super().__init__()
        self.min_similarity = 0.7
        self.history = []
        self.top_pairs = {}
        self.high_similarity_words = []
        self.words = []
        self.groups = []
        self.syns1 = []
        self.syns2 = []
        self.highest_cossim = 0

    def load_history(self, path):
        with open(path , 'r') as f:
            self.history = json.load(f)

    def get_synsets(self, word):
        return wn.synsets(word)
    
    def get_top_pairs(self, word, other_word):
        for syn1 in self.syns1:
            for syn2 in self.syns2:
                cossim = syn1.wup_similarity(syn2)
                if cossim is not None and cossim > self.highest_cossim:
                    self.highest_cossim = cossim

                if self.highest_cossim >= self.min_similarity:
                    self.top_pairs[(word, other_word)] = self.highest_cossim

    def find_high_similarity_words(self):
        self.high_similarity_words = list(set([w for pair in self.top_pairs.keys() for w in pair]))

    def find_groups(self):
        for word_group in combinations(self.high_similarity_words, 4):
            pairs = list(combinations(word_group, 2))
            if all(pair in self.top_pairs for pair in pairs):
                print("\nFound set of 4 highly similar words:", word_group)
                break
        else:
            print("\nNo set of 4 words found with high similarity across all pairs.")



In [ ]:
# call the class
sem = Semantics()
sem.load_history('datasets/history.json')

for entry in sem.history[:10]:
    sem.groups = entry['answers']       # entry is a single game
    sem.words = [word for group in sem.groups for word in group['members']]    # get all words 
    sem.words = [word.lower() for word in sem.words]
    print(sem.words)

    for i, word in enumerate(sem.words):        # iterate over all words
        sem.syns1 = sem.get_synsets(word)            # get all synsets of the word
        #print(f"\nWord '{word}' has {len(syns1)} synsets")

                
        for j, other_word in enumerate(sem.words):   # iterate over all words again
                    if i == j:
                        continue  # dont compare with same word

                    sem.syns2 = sem.get_synsets(other_word)
                    sem.highest_cossim = 0

                    # sim between synsets of word and other_word
                    sem.get_top_pairs(word, other_word)

    # Find groups of four words where all pairs have high similarity
    sem.find_high_similarity_words()
    sem.find_groups()

    

In [43]:
# phonology test
from nltk.corpus import cmudict
nltk.download('cmudict')
cmu = cmudict.dict()

for entry in history[:1]:
    groups = entry['answers']       # entry is a single game
    words = [word for group in groups for word in group['members']]    # get all words 
    words = [word.lower() for word in words]
    for word in words:
        phoneme = cmu[']

print(phoneme)



[nltk_data] Downloading package cmudict to /home/auschra/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


[['HH', 'AH0', 'L', 'OW1'], ['HH', 'EH0', 'L', 'OW1']]


In [ ]:

class Phonology():
    def __init__(self):
        super().__init__()


In [60]:
class Orthography(): # anagram, palindrome, alliteration
    def __init__(self):
        super().__init__()

# anagram (single use in archive lol)
for entry in history:
    groups = entry['answers']       # entry is a single game
    words = [word for group in groups for word in group['members']]    # get all words 
    words = [word.lower() for word in words]

    an_count = 0
    angrams = []


    palindrone_count = 0
    palindrome = []

    # never this simple
    # alliteration = []


    for i, word in enumerate(words):
        for j, other_word in enumerate(words):
            if i == j:
                continue
            if sorted(word) == sorted(other_word):
                an_count += 1
                angrams.append(other_word)
            else:
                pass

        if an_count == 4:
            print(f"Anagram found in group: {entry['id']} {angrams}")

        



Anagram found in group: 94 ['open', 'peon', 'pone', 'nepo']


In [ ]:
class PatternMatcher():
    def __init__(self):
        super().__init__()


class Morphology():
    def __init__(self):
        super().__init__()



class MultiWord():
    def __init__(self):
        super().__init__()

class Encyclopedic():
    def __init__(self):
        super().__init__()

class Association():
    def __init__(self):
        super().__init__()

class Combinations():
    def __init__(self):
        super().__init__()


In [ ]:
class ConnectionsEncoder(nn.Module):
    def __init__(self, embd=768):
        super().__init__()
            self.model = 'Llama-3-1-8B'
            self.semantic_encoder = Semantics()
            self.pattern_matcher = PatternMatcher()
            self.morphology = Morphology()
            self.orthography = Orthography()
            self.phonology = Phonology()
            self.multi_word = MultiWord()
            self.encyclopedic = Encyclopedic()
            self.association = Association()
            self.combinations = Combinations()


    def relationships(self, words):

        prompt="""
        Consider these words and their semantic relationships.
        Look for:
        1. Direct category membership
        2. Metaphorical connections
        3. Word play or double meanings
        4. Context-dependent relationships
        """

    return self.combine_evidence(semantic_embeddings, patterns)

    # beam search to find optimal groupings
    # should 1 shot the problem, don't fall for red herrings

    def combine_evidence(self, semantic_embeddings, patterns):
        # combine all evidence
        return groupings
    



In [ ]:
class BeamSearch(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = ConnectionsEncoder()

    # beam search to find optimal groupings
